In [1]:
import numpy as np

In [4]:

class Layer_Dense:
    def __init__(self, n_input, n_neurons, learning_rate=0.01):
        self.weights = np.random.randn(n_input, n_neurons) * np.sqrt(2 / n_input) #'he' initialization
        self.biases = np.zeros((1, n_neurons))
        self.learning_rate = learning_rate

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)
        
        # Update weights and biases
        self.weights -= self.learning_rate * self.dweights
        self.biases -= self.learning_rate * self.dbiases

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        
class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = (dvalues - y_true) / samples

In [3]:
X = np.array([[1, 2, 3], [0.5, -0.2, 0.1], [0.3, 0.8, -1.2]])  
y = np.array([0, 1, 2])  
dense1 = Layer_Dense(3, 5)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(5, 5)
activation2 = Activation_ReLU()

dense3 = Layer_Dense(5, 3)
activation3 = Activation_Softmax()
loss_function = Loss_CategoricalCrossentropy()

# Training loop
for epoch in range(5000):
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)
    
    loss = np.mean(loss_function.forward(activation3.output, y))
    
    # Backward pass
    loss_function.backward(activation3.output, y)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

Epoch 0, Loss: 1.0526854596020208
Epoch 100, Loss: 0.4271381419922107
Epoch 200, Loss: 0.2830556904935301
Epoch 300, Loss: 0.2066591619085143
Epoch 400, Loss: 0.15178662913429924
Epoch 500, Loss: 0.1115204514959502
Epoch 600, Loss: 0.08261911097088208
Epoch 700, Loss: 0.06228374432446801
Epoch 800, Loss: 0.047975468042636776
Epoch 900, Loss: 0.037831117779383756
Epoch 1000, Loss: 0.030496193091690507
Epoch 1100, Loss: 0.02508746571414271
Epoch 1200, Loss: 0.021008989008977717
Epoch 1300, Loss: 0.017873426281732063
Epoch 1400, Loss: 0.015414469229803461
Epoch 1500, Loss: 0.013454329217961837
Epoch 1600, Loss: 0.011865121716953484
Epoch 1700, Loss: 0.010559022704280717
Epoch 1800, Loss: 0.009473577891011897
Epoch 1900, Loss: 0.008559617881519807
Epoch 2000, Loss: 0.007783153157062561
Epoch 2100, Loss: 0.0071176397445151035
Epoch 2200, Loss: 0.006541528840712967
Epoch 2300, Loss: 0.006039982140878107
Epoch 2400, Loss: 0.005599591012823653
Epoch 2500, Loss: 0.0052111641281084925
Epoch 2600